# Подготовка ноутбука 

## Пробрасываем magic methods

In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

## Imports

In [2]:
import pandas as pd 
import numpy as np 
from data.prepare_data import PrepareData
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from torch.utils.data import DataLoader
from IPython.display import display
import pytesseract
import shutil
try:
    from PIL import Image
except ImportError:
     import Image
import cv2
import requests
import json
import os
from datetime import datetime
import csv


import pandas as pd 
import numpy as np 
from data.prepare_data import PrepareData
from dotenv import load_dotenv
import os 
from pathlib import Path
from warnings import filterwarnings 
import torch 
import torchvision
from torch.utils.data import DataLoader
from IPython.display import display
import pytesseract
import shutil
try:
    from PIL import Image
except ImportError:
     import Image
import cv2
import requests
import json
import os
from datetime import datetime
import csv
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
import hashlib
from utils.s3_optimize import S3Manager
import logging

from geo.download_images import MapillaryS3Client

## Нужные переменные 

In [3]:
# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
ROOT_DIR = Path('../../')
load_dotenv()
filterwarnings(action='ignore')

# Загрузка датафрейма

In [ ]:
def main():
    """Пример использования клиента с разбивкой bbox"""
    load_dotenv()
    
    # Инициализация S3 менеджера
    s3_manager = S3Manager(
        max_workers=8,
        chunk_size=32 * 1024 * 1024
    )
    
    # Инициализация Mapillary клиента
    mapillary_client = MapillaryS3Client(
        access_token=os.getenv('API_MAPILLARY_KEY'),
        s3_manager=s3_manager,
        max_workers=8,
        cache_dir=ROOT_DIR / 'logs/download_data/cache_moscow_images'
    )
    
    # Основной bbox для Москвы
    moscow_oblast_bbox = [36.866323, 55.334271, 38.564470, 56.191264]
    moscow_bbox = [37.366723, 55.579170, 37.841304, 55.907936]
    
    # Вариант 1: Автоматическое разбиение большого bbox
    logger.info("=== Вариант 1: Автоматическое разбиение большого bbox ===")
    images = mapillary_client.get_images_for_large_area(
        bbox=moscow_oblast_bbox,
        max_results_per_bbox=500,
        grid_size=30,
        use_cache=True
    )
    
    # Вариант 2: Сетка вокруг центральной точки
    # logger.info("=== Вариант 2: Сетка вокруг центральной точки ===")
    # moscow_center = (55.7558, 37.6173)  # Кремль
    # images = mapillary_client.get_images_around_point(
    #     center_lat=moscow_center[0],
    #     center_lon=moscow_center[1],
    #     grid_radius=3,  # 7x7 = 49 bbox
    #     bbox_size=0.01,  # Размер каждого bbox
    #     max_results_per_bbox=300
    # )
    
    if images:
        logger.info(f"Найдено {len(images)} изображений. Начинаем загрузку в S3...")
        
        # Загружаем в S3
        start_time = time.time()
        results = mapillary_client.download_and_upload_to_s3(
            images=images,
            s3_prefix="site/raw_data",
            image_format="JPEG", 
            quality=80
        )
        
        duration = time.time() - start_time
        logger.info(f"Загрузка завершена за {duration:.2f} секунд")
        
        # Сохраняем метаданные
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        mapillary_client.save_metadata_to_csv(
            images, 
            ROOT_DIR / 'data/processed_data/moscow_images.csv',
        )


if __name__ == "__main__":
    main()

INFO:__main__:=== Вариант 1: Автоматическое разбиение большого bbox ===
INFO:geo.download_images:Основной bbox разбит на 900 частей
INFO:geo.download_images:  Часть 1: [36.866323, 55.334271, 36.922927900000005, 55.36283743333333]
INFO:geo.download_images:  Часть 2: [36.866323, 55.36283743333333, 36.922927900000005, 55.391403866666664]
INFO:geo.download_images:  Часть 3: [36.866323, 55.391403866666664, 36.922927900000005, 55.4199703]
INFO:geo.download_images:  Часть 4: [36.866323, 55.4199703, 36.922927900000005, 55.448536733333334]
INFO:geo.download_images:  Часть 5: [36.866323, 55.448536733333334, 36.922927900000005, 55.477103166666666]
INFO:geo.download_images:  Часть 6: [36.866323, 55.477103166666666, 36.922927900000005, 55.5056696]
INFO:geo.download_images:  Часть 7: [36.866323, 55.5056696, 36.922927900000005, 55.534236033333336]
INFO:geo.download_images:  Часть 8: [36.866323, 55.534236033333336, 36.922927900000005, 55.56280246666667]
INFO:geo.download_images:  Часть 9: [36.866323, 

In [ ]:
s3_manager = S3Manager(
        max_workers=4,
        chunk_size=16 * 1024 * 1024
    )

s3_files = s3_manager.list_files(prefix="site/raw_data/", file_extensions=[".jpg", ".jpeg", ".png"])
download_results = s3_manager.batch_download_files(
    s3_files,
    local_dir=ROOT_DIR / 'data/processed_data/moscow_image',
    progress_callback=lambda current, total: print(f"Download progress: {current}/{total}")
)

INFO:utils.s3_optimize:Успешно подключились к S3 бакету: s3-dvc


Download progress: 10/85840
Download progress: 20/85840
Download progress: 30/85840
Download progress: 40/85840
Download progress: 50/85840
Download progress: 60/85840
Download progress: 70/85840
Download progress: 80/85840
Download progress: 90/85840
Download progress: 100/85840
Download progress: 110/85840
Download progress: 120/85840
Download progress: 130/85840
Download progress: 140/85840
Download progress: 150/85840
Download progress: 160/85840
Download progress: 170/85840
Download progress: 180/85840
Download progress: 190/85840
Download progress: 200/85840
Download progress: 210/85840
Download progress: 220/85840
Download progress: 230/85840
Download progress: 240/85840
Download progress: 250/85840
Download progress: 260/85840
Download progress: 270/85840
Download progress: 280/85840
Download progress: 290/85840
Download progress: 300/85840
Download progress: 310/85840
Download progress: 320/85840
Download progress: 330/85840
Download progress: 340/85840
Download progress: 350/